<a href="https://colab.research.google.com/github/Melisa7L/PPS/blob/main/pipeline(clasificacion_de_urls).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

librerias

In [2]:
pip install requests

In [3]:
pip install beautifulsoup4

In [4]:
pip install pandas

In [6]:
pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 2.9 MB/s eta 0:00:00


*Codigo donde se colocan las urls que se quieren analizar y de ahi comienza el proceso de clasificarlas de acuerdo a palabras clave y el contenido de las mismas*
Al final de la clasificacion se genera un excel que contiene los datos clasificados y ordenados en 5 columnas :'fecha_noticia', 'titulo_noticia', 'texto_noticia', 'longitud_noticia', 'categoria' .

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Lista de URLs para scrapear son algunas de ejemplo
urls = [
    'https://www.c5n.com/deportes/mundial-sub-17-argentina-cayo-frente-alemania-y-jugara-el-tercer-puesto-n136601',
    'https://www.c5n.com/politica/javier-milei-se-reune-jake-sullivan-asesor-seguridad-nacional-joe-biden-n136598',
    'https://www.c5n.com/economia/dolar-blue-hoy-cuanto-cotiza-el-28-noviembre-n136604',
    'https://www.c5n.com/astrologia/horoscopo-hoy-martes-28-noviembre-n136580',
    'https://www.c5n.com/tecnologia/alerta-android-una-app-mensajes-espia-tus-datos-n136409',
    'https://www.c5n.com/ratingcero/emotivo-taylor-swift-recibio-la-familia-la-fanatica-que-murio-brasil-n136557'
]

# Palabras clave a buscar en las URLs
keywords = ['article', 'noticias', 'news', 'nota', '/']

# Algunas palabras clave para cada categoría
palabras_clave = {
    'salud': ['salud', 'medicina', 'enfermedad', 'medico', 'bienestar'],
    'deportes': ['deportes', 'futbol', 'baloncesto', 'tenis', 'competicion'],
    'medioambiente': ['medioambiente', 'ecologia', 'naturaleza', 'sostenibilidad', 'cambio climatico'],
    'economia': ['economia', 'finanzas', 'negocios', 'mercado', 'inversion'],
    'politica': ['politica', 'gobierno', 'elecciones', 'legislación', 'parlamento'],
    'entretenimiento': ['entretenimiento', 'cine', 'musica', 'celebridades', 'espectaculos'],
    'horoscopo': ['horoscopo', 'astrologia', 'signos zodiacales', 'predicciones', 'astrologia diaria'],
    'cultura': ['cultura', 'arte', 'literatura', 'tradiciones', 'patrimonio'],
    'ciencia_tecnologia': ['ciencia', 'tecnologia', 'innovacion', 'descubrimientos', 'avances cientificos'],
    'educacion': ['educacion', 'escuela', 'aprendizaje', 'docentes', 'formacion']
}

# Creacion de listas para almacenar datos
fechas = []
textos = []
titulos = []
categorias = []

# Función para determinar la categoría de la noticia
def determinar_categoria(texto_noticia):
    for categoria, palabras in palabras_clave.items():
        for palabra in palabras:
            if palabra in texto_noticia.lower():
                return categoria
    return 'Otra'

# Función para obtener los datos de una URL
def scrape_noticia(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extrae el título de la noticia
    titulo_noticia = soup.find('h1').text.strip() if soup.find('h1') else "no especificado"

    # Extrae el texto de la noticia
    texto_noticia = ' '.join([p.text.strip() for p in soup.find_all('p')])

    # Determina la categoría de la noticia
    categoria = determinar_categoria(texto_noticia)

    # Para extraer la fecha de la noticia
    fecha_noticia = soup.find(['time', 'span', 'p', 'publication-date', 'fecha'], {'datetime': True})
    fecha_noticia = fecha_noticia['datetime'].strip() if fecha_noticia else "Fecha no encontrada"

    fechas.append(fecha_noticia)
    textos.append(texto_noticia)
    titulos.append(titulo_noticia)
    categorias.append(categoria)

# Iterar a través de las urls y extraer los datos de cada una
for url in urls:
    scrape_noticia(url)

# Crear un DataFrame con los datos recopilados anteriormente
data = {
    'fecha_noticia': fechas,
    'titulo_noticia': titulos,
    'texto_noticia': textos,
    'categoria': categorias
}

df = pd.DataFrame(data)

# Calcular la longitud de la noticia
df['longitud_noticia'] = df['texto_noticia'].apply(len)

# Se ordenan las columnas en este orden que elegi
df = df[['fecha_noticia', 'titulo_noticia', 'texto_noticia', 'longitud_noticia', 'categoria']]

# Ordenar el DataFrame por 'categoria' y 'fecha_noticia'
df = df.sort_values(by=['categoria', 'fecha_noticia'])

# Guardar el DataFrame en un archivo Excel (.xlsx)
output_file = 'Data.xlsx'
df.to_excel(output_file, index=False, engine='xlsxwriter')

# Imprimir el DataFrame
print(df)

# Imprimir mensaje de confirmación
print(f"Los datos se han guardado en '{output_file}'.")


              fecha_noticia  \
5  2023-11-27T21:14:44.000Z   
0  2023-11-28T10:27:00.000Z   
2  2023-11-28T11:16:08.000Z   
4  2023-11-27T13:57:17.000Z   
1  2023-11-28T10:33:00.000Z   
3  2023-11-27T23:58:55.000Z   

                                      titulo_noticia  \
5  Emotivo: Taylor Swift recibió a la familia de ...   
0  Mundial Sub 17: Argentina cayó frente a Aleman...   
2  Dólar blue hoy: a cuánto cotiza el 28 de novie...   
4  Alerta Android: una app de mensajes espía tus ...   
1  Javier Milei se reúne Jake Sullivan, asesor de...   
3           Horóscopo de hoy, martes 28 de noviembre   

                                       texto_noticia  longitud_noticia  \
5  Ana Clara Benavides falleció en la previa al p...              2265   
0  Los dirigidos por Diego Placente lograron igua...              4071   
2  El tipo de cambio informal opera a $999 en el ...              3277   
4  La aplicación fue retirada de la tienda oficia...              3316   
1  El presidente el